In [1]:
import numpy as np
import pandas as pd 

In [2]:
df=pd.read_csv('ratings.csv')

In [3]:
df.head()

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510


In [4]:
df2=pd.read_csv('movies.csv')

In [5]:
df2.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [6]:
df2.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [7]:
df2.isnull().sum()

movieId    0
title      0
genres     0
dtype: int64

In [8]:
merged_df = pd.merge(df, df2, on='movieId', how='inner')


In [9]:
print(merged_df.head())

   userId  movieId  rating   timestamp  \
0       1      296     5.0  1147880044   
1       1      306     3.5  1147868817   
2       1      307     5.0  1147868828   
3       1      665     5.0  1147878820   
4       1      899     3.5  1147868510   

                                              title  \
0                               Pulp Fiction (1994)   
1  Three Colors: Red (Trois couleurs: Rouge) (1994)   
2  Three Colors: Blue (Trois couleurs: Bleu) (1993)   
3                                Underground (1995)   
4                        Singin' in the Rain (1952)   

                        genres  
0  Comedy|Crime|Drama|Thriller  
1                        Drama  
2                        Drama  
3             Comedy|Drama|War  
4       Comedy|Musical|Romance  


In [10]:
print(merged_df[['title', 'genres']].head())


                                              title  \
0                               Pulp Fiction (1994)   
1  Three Colors: Red (Trois couleurs: Rouge) (1994)   
2  Three Colors: Blue (Trois couleurs: Bleu) (1993)   
3                                Underground (1995)   
4                        Singin' in the Rain (1952)   

                        genres  
0  Comedy|Crime|Drama|Thriller  
1                        Drama  
2                        Drama  
3             Comedy|Drama|War  
4       Comedy|Musical|Romance  


In [11]:
print(type(merged_df))


<class 'pandas.core.frame.DataFrame'>


In [12]:
print(merged_df.columns)


Index(['userId', 'movieId', 'rating', 'timestamp', 'title', 'genres'], dtype='object')


In [13]:
merged_df.isnull().sum()

userId       0
movieId      0
rating       0
timestamp    0
title        0
genres       0
dtype: int64

In [14]:
merged_df.isnull().sum()

userId       0
movieId      0
rating       0
timestamp    0
title        0
genres       0
dtype: int64

In [15]:
from sklearn.preprocessing import MultiLabelBinarizer


In [16]:
sample_df = merged_df.head(1000)  

sample_df['genre_list'] = sample_df['genres'].apply(lambda x: x.split('|') if isinstance(x, str) else [])
mlb = MultiLabelBinarizer()
genre_encoded = mlb.fit_transform(sample_df['genre_list'])
genre_df = pd.DataFrame(genre_encoded, columns=mlb.classes_)
final_df = pd.concat([sample_df.drop(columns=['genres', 'genre_list']), genre_df], axis=1)

print(final_df.head())




   userId  movieId  rating   timestamp  \
0       1      296     5.0  1147880044   
1       1      306     3.5  1147868817   
2       1      307     5.0  1147868828   
3       1      665     5.0  1147878820   
4       1      899     3.5  1147868510   

                                              title  (no genres listed)  \
0                               Pulp Fiction (1994)                   0   
1  Three Colors: Red (Trois couleurs: Rouge) (1994)                   0   
2  Three Colors: Blue (Trois couleurs: Bleu) (1993)                   0   
3                                Underground (1995)                   0   
4                        Singin' in the Rain (1952)                   0   

   Action  Adventure  Animation  Children  ...  Film-Noir  Horror  IMAX  \
0       0          0          0         0  ...          0       0     0   
1       0          0          0         0  ...          0       0     0   
2       0          0          0         0  ...          0       0     0

C:\Users\Aman Tiwari\AppData\Local\Temp\ipykernel_22556\834557146.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_df['genre_list'] = sample_df['genres'].apply(lambda x: x.split('|') if isinstance(x, str) else [])


In [18]:
from sklearn.metrics.pairwise import cosine_similarity

similarity=cosine_similarity(genre_df)
print(similarity)

[[1.         0.5        0.5        ... 0.35355339 0.57735027 0.25      ]
 [0.5        1.         1.         ... 0.         0.57735027 0.        ]
 [0.5        1.         1.         ... 0.         0.57735027 0.        ]
 ...
 [0.35355339 0.         0.         ... 1.         0.         0.35355339]
 [0.57735027 0.57735027 0.57735027 ... 0.         1.         0.        ]
 [0.25       0.         0.         ... 0.35355339 0.         1.        ]]


In [21]:
movie_name = 'Conjuring,The(2013)'


matches = merged_df[merged_df['title'].str.strip().str.lower() == movie_name.strip().lower()]

if not matches.empty:
    movie_index = matches.index[0]
    similar_movies = list(enumerate(similarity[movie_index]))
    similar_movies = sorted(similar_movies, key=lambda x: x[1], reverse=True)

    for i in similar_movies[1:6]:
        print(merged_df.iloc[i[0]]['title'])
else:
    print(f"❌ Movie '{movie_name}' not found in the dataset.")


❌ Movie 'Conjuring,The(2013)' not found in the dataset.


In [19]:
final_df.head()

,userId,movieId,rating,timestamp,title,(no genres listed),Action,Adventure,Animation,Children,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,296,5.0,1147880044,Pulp Fiction (1994),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,1,306,3.5,1147868817,Three Colors: Red (Trois couleurs: Rouge) (1994),0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,307,5.0,1147868828,Three Colors: Blue (Trois couleurs: Bleu) (1993),0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,665,5.0,1147878820,Underground (1995),0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,1,899,3.5,1147868510,Singin' in the Rain (1952),0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0


In [20]:
final_df.sample()

,userId,movieId,rating,timestamp,title,(no genres listed),Action,Adventure,Animation,Children,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
959,4,4709,2.0,1573942459,Paint Your Wagon (1969),0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
